In [1]:
!pip install datasets
!pip install transformers
!pip install einops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 8.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 32.5 MB/s eta 0:00

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'deeplearning/'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

Mounted at /content/drive
['shortjokes.csv', 'openseminar_GAN.pptm', 'openseminar_GAN.pptm.gslides', 'Untitled0.ipynb', 'openseminar_GAN_final.docx', 'Davinci003.ipynb', 'Untitled', 'GPTNEO.ipynb', 'baseline.ipynb', 'RM_Training.ipynb', 'trained_models', 'Transformers.ipynb', 'QA_RL_PPO.ipynb', 'api.ipynb', 'RLHF_Dataset (1).ipynb', 'GPTNeoMyself.ipynb', 'InstructMyself2.ipynb', 'q-learning.ipynb', 'deepdai', 'VAE.ipynb', 'diffusion', 'competition', 'GPTNeo_RLHF.ipynb', 'CRS', 'reward_logs.txt', 'InstructMyself.ipynb']


In [56]:
from datasets import load_dataset

# Load all helpfulness/harmless subsets (share the same schema)
train_dataset = load_dataset("Anthropic/hh-rlhf", split="train")
test_dataset= load_dataset("Anthropic/hh-rlhf", split="test")

dialogue_dataset = load_dataset("daily_dialog")

  0%|          | 0/3 [00:00<?, ?it/s]

In [57]:
import pandas as pd
import re
from itertools import combinations
import io
import json
import os
import ast

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions.categorical import Categorical
from tqdm import tqdm

import transformers
from transformers import AutoTokenizer, BertForSequenceClassification
# from transformers import GPTNeoForCausalLM, GPT2Tokenizer, AdamW, get_scheduler, GPTNeoModel, GPT2LMHeadModel
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from typing import List, Deque
from einops import rearrange

In [58]:
import logging
logging.getLogger().setLevel(logging.CRITICAL)

import warnings
warnings.filterwarnings('ignore')

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [59]:
model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-yelp-polarity", bos_token='<|startoftext|>', eos_token='<|endoftext|>', padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

models_folder = "trained_models" # where to save model

model.to(device)
print('skip')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


skip


In [60]:
prompts=["Hello it's me."]

In [61]:
tokenized=tokenizer(prompts, return_tensors='pt', truncation=True, padding=True) # tokenize 후 pytorch tensor를 return 하라는 뜻
print("ㄴ")
input_ids, attention_mask = tokenized.input_ids.to(device), tokenized.attention_mask.to(device)
print("ㅇㅇ", input_ids.shape)

ㄴ
ㅇㅇ torch.Size([1, 8])


In [62]:
last_hidden_states=model(input_ids, attention_mask=attention_mask)

In [63]:
last_hidden_states

SequenceClassifierOutput(loss=None, logits=tensor([[ 1.4897, -1.7411]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [64]:
import copy 

class RmDataset(torch.utils.data.Dataset):
  def __init__(self):
    super().__init__()

    self.chosen_list = []
    self.rejected_list = []
    self.end_of_text_token = "<|endoftext|>"
    
    x = 0
    train_dataset_copy = copy.deepcopy(train_dataset)
    for td in train_dataset_copy:
      text1 = td['chosen'].split("Assistant:")[-1]
      text2 = td['rejected'].split("Assistant:")[-1]
      if len(text1)<500 and len(text2)<500:
        self.chosen_list.append(f"{text1}")
        self.rejected_list.append(f"{text2}")

  def __len__(self):
    return len(self.chosen_list)

  def __getitem__(self, index):
    chosen_text = self.chosen_list[index]
    rejected_text = self.rejected_list[index]

    return chosen_text, rejected_text

In [65]:
class BERTSequenceClassificationRewardModel(nn.Module):
  def __init__(self):
    super(BERTSequenceClassificationRewardModel, self).__init__()
    self.base = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity").to(device)
    self.tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", bos_token='<|startoftext|>', eos_token='<|endoftext|>', padding_side="left")
    self.tokenizer.pad_token = '<|endoftext|>'
  
  def forward(self, prompts):
    # prompts include both state(모델에서 주어진 상황: 질문) and action(답변: 모델의 행동 -> 답변) could be raw text of tensor
    # ppo 할 때 얘기
    print("ㄲ", [prompts[0]])
    tokenized=self.tokenizer(prompts, return_tensors='pt', truncation=True, padding=True) # tokenize 후 pytorch tensor를 return 하라는 뜻
    print("ㄴ")
    input_ids, attention_mask = tokenized.input_ids.to(device), tokenized.attention_mask.to(device)
    print("ㅇㅇ", input_ids.shape)
    last_hidden_states=self.base(input_ids, attention_mask=attention_mask)
    print("ㅇㅇss", last_hidden_states)
    
    # use final hidden state's EOS embedding token for classification
    return last_hidden_states

In [66]:
reward_model = BERTSequenceClassificationRewardModel()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [67]:
class RMTrainer():
  def __init__(self, data):
    self.train_data=data['train_data']
    self.val_data=data['val_data']
    # self.test_data=data['test_data']
    self.sigmoid = nn.Sigmoid()
    #training logs
    self.train_logs={ #loss값 기록용
        "loss_history":[],
        "acc_history":[],
        "val_acc_history":[],
    }
    self.reward_model = reward_model

  def plot_logs(self):
    loss_history=self.train_logs['loss_history']
    acc_history=self.train_logs['acc_history']
    plt.figure(figsize=(15,5))
    plt.xlabel("Iteration")
    plt.ylabel("Loss")
    plt.plot(loss_history, "b-")
    plt.show()

    plt.figure(figsize=(15,5))
    plt.xlabel("Iteration")
    plt.ylabel("Accuracy")
    plt.plot(acc_history, "r-")
    plt.show()

  def get_loss(self, chosens, rejecteds):
    # 전부 다 배치로 처리된다는 점을 고려해야한다.
    batch_size = len(rejecteds)
    correct_count=0
    print(11, chosens)
    
    chosen_rewards, eos = self.reward_model(chosens) # should be +
    print(22)
    rejecteds_rewards, eos = self.reward_model(rejecteds) # should be -

    loss = torch.FloatTensor([0]).to(device)
    print(33)

    for idx, rejec_reward in enumerate(rejecteds_rewards):
      chosen_reward = chosen_rewards[idx]
      diff = chosen_reward - rejec_reward # this should be +1,    -1 is the worst
      loss = loss-torch.log(self.sigmoid(diff))
      correct_count+=(chosen_reward.item()>rejec_reward.item()) # for count
    print(44)

    loss = loss/len(rejecteds_rewards)
    accuracy = correct_count/len(rejecteds_rewards)
    return loss, accuracy

  def train(self, num_epochs, batch_size, lr, reg):
    train_loader = torch.utils.data.DataLoader(self.train_data, batch_size=batch_size, shuffle=True)
    rm_optimizer = optim.AdamW(self.reward_model.parameters(), lr=lr, weight_decay=reg)
    print(1)
    
    for epoch in range(num_epochs):
      print(5)
      train_acc = 0
      for idx, data in tqdm(enumerate(train_loader)):
        print(6)
        self.reward_model.train() # train 상태로 둔다.
        print(2)
        chosens, rejecteds = data
        loss, acc = self.get_loss(chosens, rejecteds)
        print(3)

        # weight update
        rm_optimizer.zero_grad()
        loss.backward()
        rm_optimizer.step()

        # tran_acc update
        print(4)
        train_acc = (train_acc*idx + acc)/(idx+1)
        self.train_logs['loss_history'].append(loss.item())
        self.train_logs['acc_history'].append(train_acc)

        if idx%200 == 0:
          self.plot_logs()
          print(f"Loss: {np.mean(self.train_logs['loss_history'])}, all_acc : {train_acc}")
      val_acc=self.validate()
      print("validation accuracy : ", val_acc)

In [68]:
reward_dataset = RmDataset()
dataset_size = len(reward_dataset)

train_size = int(dataset_size * 0.8)
validation_size = dataset_size - train_size
# test_size = dataset_size - train_size - validation_size

train_dataset, validation_dataset = torch.utils.data.random_split(reward_dataset, [train_size, validation_size])

splited_data = {"train_data":train_dataset, "val_data":validation_dataset}

In [69]:
anth = RMTrainer(splited_data)

In [70]:
setting = {
    "num_epochs":4,
    "batch_size":1,
    "lr":0.00012, # learning rate
    "reg":0.008,
}

In [71]:
anth.train(setting['num_epochs'], setting['batch_size'], setting['lr'], setting['reg'])

1
5


0it [00:00, ?it/s]

6
2
11 (' Hello, I’m your virtual assistant. Are you looking for Rick Springfield albums? Here’s the list:',)
ㄲ [' Hello, I’m your virtual assistant. Are you looking for Rick Springfield albums? Here’s the list:']
ㄴ
ㅇㅇ torch.Size([1, 25])


ValueError: ignored